Loading and starting ib_insync

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=3)

<IB connected to 127.0.0.1:7496 clientId=3>

Loading and starting the Redis database.

The following line runs a script that contains the following two lines:
1. import redis
2. red = redis.Redis(password='mypassword')

In [2]:
%run redis_start.py

Reading in the current ask price from the Redis database (after setting the type to float).

In [4]:
red.set_response_callback('GET',float)
spy_bid = red.get('SPY_bid')
spy_ask = red.get('SPY_ask')
print('spy bid price is', spy_bid)
print('spy ask price is', spy_ask)

spy bid price is 359.81
spy ask price is 359.83


Buying 100 shares using a limit order.  Since the limit price is above the current ask, this should get filled immediately.

In [5]:
contract = Stock('SPY', 'SMART', 'USD')
order = LimitOrder('BUY', 1, spy_ask+.25)
order.outsideRth = True
ib.placeOrder(contract, order)

Trade(contract=Stock(symbol='SPY', exchange='SMART', currency='USD'), order=LimitOrder(orderId=41, clientId=3, action='BUY', totalQuantity=1, lmtPrice=360.08, outsideRth=True, conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 11, 9, 16, 48, 32, 660455, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

In [6]:
ib.sleep(1)

True

Now putting in a limit order to buy 100 shares.  Since the limit price is much lower, this order will not be filled immediately, if ever. 

In [7]:
contract = Stock('SPY', 'SMART', 'USD')
order = LimitOrder('BUY', 100, spy_bid-10)
order.outsideRth = True
ib.placeOrder(contract, order)

Trade(contract=Stock(symbol='SPY', exchange='SMART', currency='USD'), order=LimitOrder(orderId=42, clientId=3, action='BUY', totalQuantity=100, lmtPrice=349.81, outsideRth=True, conditions=[], softDollarTier=SoftDollarTier()), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 11, 9, 16, 48, 51, 842923, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

We should be able to see that the previous order is still open, meaning that our order has not been filled.

In [8]:
openOrders = ib.openOrders()
print(openOrders)

[LimitOrder(orderId=42, clientId=3, permId=1710700177, action='BUY', totalQuantity=100.0, lmtPrice=349.81, auxPrice=0.0, tif='DAY', ocaType=3, outsideRth=True, trailStopPrice=350.81, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU1457230', clearingIntent='IB', orderComboLegs=[], adjustedOrderType='None', conditions=[], softDollarTier=SoftDollarTier(), cashQty=0.0, dontUseAutoPriceForHedge=True)]


If we execute the following line, we will see our order cancelled.

In [9]:
ib.cancelOrder(openOrders[0])

Trade(contract=Stock(symbol='SPY', exchange='SMART', currency='USD'), order=LimitOrder(orderId=42, clientId=3, permId=1710700177, action='BUY', totalQuantity=100.0, lmtPrice=349.81, auxPrice=0.0, tif='DAY', ocaType=3, outsideRth=True, trailStopPrice=350.81, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU1457230', clearingIntent='IB', orderComboLegs=[], adjustedOrderType='None', conditions=[], softDollarTier=SoftDollarTier(), cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='PendingCancel', remaining=100.0, permId=1710700177, clientId=3), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 11, 9, 16, 48, 51, 842923, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2020, 11, 9, 16, 48, 51, 887307, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2020, 11, 9, 16, 49, 26, 444474, tzinfo=datetime.timezone.utc), status=

We can also see all our current positions:

In [10]:
ib.positions()

[Position(account='DU1457230', contract=Forex('EURUSD', conId=12087792, localSymbol='EUR.USD', tradingClass='EUR.USD'), position=-20.0, avgCost=0.892755),
 Position(account='DU1457230', contract=Stock(conId=70282872, symbol='MINT', exchange='ARCA', currency='USD', localSymbol='MINT', tradingClass='MINT'), position=200.0, avgCost=101.72),
 Position(account='DU1457230', contract=Stock(conId=265598, symbol='AAPL', exchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), position=68.0, avgCost=51.64201325),
 Position(account='DU1457230', contract=Stock(conId=208813720, symbol='GOOG', exchange='NASDAQ', currency='USD', localSymbol='GOOG', tradingClass='NMS'), position=10.0, avgCost=1215.81),
 Position(account='DU1457230', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=101.0, avgCost=309.98871287128713)]

This will disconnect from IB.

In [11]:
ib.disconnect()